In [ ]:
from pathlib import Path
import pandas as pd
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

In [ ]:
analysis_version = "2017_10_19"
project_dir = Path('/Users/rodgersleejg/data/hpc/NNDSP') # needs to be pathlib.Path object

In [ ]:
%pwd
%cd {project_dir}
%pwd

In [ ]:
bids_dir = project_dir.joinpath('bids_2017_07_14')

mriqc_dir  = project_dir.joinpath('anal/mriqc_files/other_files')
if not mriqc_dir.exists():
    mriqc_dir.mkdir()
output_folder =  project_dir / 'derivatives' / 'mriqc'
if not output_folder.exists():
    output_folder.mkdir()

log_dir = mriqc_dir.joinpath('swarm_output_' +  analysis_version)
if not log_dir.exists():
    log_dir.mkdir()
manual_qc = output_folder.joinpath('manual_qc_metrics.csv')
second_round_mprage_ratings = mriqc_dir.joinpath('ohbm_2018','mprage_multi_scans.csv')
out_qc = manual_qc.with_name('manual_qc_round_2.tsv')

In [ ]:
df_manual = pd.read_csv(manual_qc)
difficult_col1 = [c for c in df_manual.columns if c.find('Freesurfer_int')==0 and c.find('discrep')>0][0]
difficult_col2 = 'Freesurfer_avg_int_rating '
df_manual = (df_manual.
             rename(columns = 
                    {
                        difficult_col1 : 'Freesurfer_int_discrepency',
                        difficult_col2 : 'Freesurfer_avg_int_rating'
                    }
                    
                   )
            )
df_manual['MASKID'] = df_manual.MASKID.apply(lambda x: '{n:04d}'.format(n = x))
df_manual['run'] = '001'
df_manual.head()

In [ ]:
df_addition = pd.read_csv(second_round_mprage_ratings).rename(columns = {'Mask ID':'MASKID'})
df_addition['MASKID'] = df_addition.MASKID.apply(lambda x: '{n:04d}'.format(n = x))
df_addition['Notes'] = df_addition.Notes.str.replace('(\d/\d/2014)','date')
df_addition.head()

In [ ]:
patterns_1 = ['first','1st','1/','1 of','3TA']
patterns_2 = ['second','2nd','2/','2 of','3TB']
patterns_3 = ['third','3rd','3/','3 of','reliability']
patterns_4 = ['fourth','4th','4/','4 of']
p1 = '.*' + '.*|.*'.join(patterns_1) + '.*'
p2 = '.*' + '.*|.*'.join(patterns_2) + '.*'
p3 = '.*' + '.*|.*'.join(patterns_3) + '.*'
p4 = '.*' + '.*|.*'.join(patterns_4) + '.*'


df_addition.loc[ df_addition.Notes.str.contains(p1,regex=True), 'run'] = '001'
df_addition.loc[ df_addition.Notes.str.contains(p2,regex=True), 'run'] = '002'
df_addition.loc[ df_addition.Notes.str.contains(p3,regex=True), 'run'] = '003'
df_addition.loc[ df_addition.Notes.str.contains(p4,regex=True), 'run'] = '004'
df_addition.loc[df_addition.run.isnull(),:]

In [ ]:
df_addition.loc[ df_addition.Notes.str.contains('.*3TB.*',regex=True), :]

In [ ]:
df_addition.head()

# Merging the data

In [ ]:
addition_ids = df_addition.MASKID.unique()
addition_ids


In [ ]:
cols_kept = ['MASKID', 'nuclear_fam_id', 'Sex', 'age_at_scan']
df_matching = (df_manual.
               loc[[iii in addition_ids for iii in df_manual.MASKID],cols_kept].
               merge(df_addition.drop('Scanner', axis =1).rename(columns = {'QC':'MPRAGE'}),
                     on = 'MASKID',
                     indicator = True).
               drop('_merge',axis = 1)
              )

df_matching is df_manual and df_addition merged. Need to remove these ids from df_manual and then merge back into that.


In [ ]:
df_full = (df_manual.
           loc[[iii not in addition_ids for iii in df_manual.MASKID],:].
           merge(df_matching,indicator = True,how = 'outer')
          )

In [ ]:
df_full.groupby('_merge').count()

In [ ]:
print(len(df_manual),len(df_addition),len(df_full))

In [ ]:
assert len(df_full.query("_merge == 'both'")) == 0
print(len(df_full.query("_merge == 'both'")))
df_full = df_full.drop('_merge',axis = 1).sort_values(['MASKID','run']).reset_index(drop = True)




In [ ]:

df_full.to_csv(out_qc,sep = '\t',index = False)
df_full.head()

In [ ]:
out_qc